# LU decomposition - pivoting

The previous notebook presents an algorithm for computing the [LU decomposition](https://en.wikipedia.org/wiki/LU_decomposition). The algotithm, however, does not apply any pivoting scheme for preventing spurious errors due to numerical instabilities. The algorithm presented here, on the other hand, shows how the partial pivoting changes our previous algorithm for computing the LU decomposition.

Let's recall the solution of a linear system by using the Gaussian elimination with partial pivoting:

$$
\begin{array}{ccccc}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \\\\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{P}^{(1)}\mathbf{y}^{(0)} \\\\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{P}^{(2)}\mathbf{y}^{(1)}
\end{array} \: ,
$$

where $\mathbf{P}^{(k)}$ is the permutation matrix used to interchange the rows and perform the partial pivoting.

At the end of this algorithm, the original matrix $\mathbf{A}$ is transformed into an upper triangular matrix $\mathbf{A}^{(N-1)}$, where, in this case, $N = 3$.

Notice that, according to the algorithm, the matrix $\mathbf{A}^{(2)}$ can be written as follows:

$$
\begin{split}
\mathbf{A}^{(2)} 
&= 
\left( \mathbf{I} - \mathbf{M}^{(2)} \right) \mathbf{P}^{(2)} \,
\left( \mathbf{I} - \mathbf{M}^{(1)} \right) \mathbf{P}^{(1)} \,
\mathbf{A} \\
&=
\underbrace{\left( \mathbf{I} - \mathbf{M}^{(2)} \right)}_{\tilde{\mathbf{Q}}^{(2)}} \,
\underbrace{\mathbf{P}^{(2)} \left( \mathbf{I} - \mathbf{M}^{(1)} \right) \mathbf{P}^{(-2)}}_{\tilde{\mathbf{Q}}^{(1)}} \,
\underbrace{\mathbf{P}^{(2)} \mathbf{P}^{(1)}}_{\mathbf{P}} \,
\mathbf{A} \\
&= \tilde{\mathbf{Q}} \mathbf{P} \mathbf{A}
\end{split} \: ,
$$

where $\mathbf{P}^{(-k)} \equiv \left( \mathbf{P}^{(k)} \right)^{-1}$, $\mathbf{P} = \mathbf{P}^{(N-1)}\mathbf{P}^{(N-2)} \cdots  \mathbf{P}^{(1)}$, $\tilde{\mathbf{Q}} = \tilde{\mathbf{Q}}^{(N-1)} \tilde{\mathbf{Q}}^{(N-2)} \cdots \tilde{\mathbf{Q}}^{(1)}$, $N = 3$. It is worth noting the diference between the matrix $\tilde{\mathbf{Q}}$ presented here and the matrix $\mathbf{Q}$ presented in the previous notebook.

The permuted matrix $\mathbf{P}\mathbf{A}$ can be written as follows:

$$
\mathbf{P}\mathbf{A} = \tilde{\mathbf{Q}}^{-1} \, \mathbf{A}^{(2)} \: ,
$$

where $\tilde{\mathbf{Q}}^{-1} = \tilde{\mathbf{Q}}^{(-1)} \tilde{\mathbf{Q}}^{(-2)}$, $\tilde{\mathbf{Q}}^{(-k)} \equiv \left( \tilde{\mathbf{Q}}^{(k)} \right)^{-1}$. As in the previous class, we need to know how to define a generic $\tilde{\mathbf{Q}}^{(-k)}$. Let's first define a generic $\tilde{\mathbf{Q}}^{(k)}$ as follows:

$$
\tilde{\mathbf{Q}}^{(k)} = 
\tilde{\mathbf{P}}^{(k)} \, \left( \mathbf{I} - \mathbf{M}^{(k)} \right) \, \tilde{\mathbf{P}}^{(-k)} \: ,
$$

where

$$\tilde{\mathbf{P}}^{(k)} = \mathbf{P}^{(N-1)} \cdots \mathbf{P}^{(k+2)}\mathbf{P}^{(k+1)} \: .$$

Then, a generic $\tilde{\mathbf{Q}}^{(-k)}$ is given by:

$$
\tilde{\mathbf{Q}}^{(-k)} = \tilde{\mathbf{P}}^{(k)} \,
\left( \mathbf{I} + \mathbf{M}^{(k)} \right) \,
\tilde{\mathbf{P}}^{(-k)} \: .
$$

It can be shown that the permutation matrices $\mathbf{P}^{(k)}$ satisfy the equation $\mathbf{P}^{(-k)} = \left( \mathbf{P}^{(k)} \right)^{\top}$, which means that they are [orthogonal matrices](http://mathworld.wolfram.com/OrthogonalMatrix.html). Consequently, the matrices $\tilde{\mathbf{P}}^{(k)}$ are orthogonal as well. The cell below shows an example of permutation matrix constructed by randomly shuffling the rows of an identity matrix. The routine [`numpy.random.shuffle`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.shuffle.html) is used.

In [1]:
import numpy as np

In [2]:
p = np.arange(10)

In [3]:
print p

[0 1 2 3 4 5 6 7 8 9]


In [4]:
np.random.shuffle(p)

In [5]:
print p

[5 2 0 8 7 4 3 1 9 6]


In [6]:
P = np.identity(p.size)[p]

In [7]:
print P

[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [8]:
np.allclose(np.dot(P.T,P), np.identity(p.size))

True

Now, let's analyze the matrix $\tilde{\mathbf{Q}}^{(-k)}$:

$$
\begin{split}
\tilde{\mathbf{Q}}^{(-k)} &= \tilde{\mathbf{P}}^{(-k)} \,
\left( \mathbf{I} + \mathbf{M}^{(k)} \right) \,
\tilde{\mathbf{P}}^{(k)} \\
&= \mathbf{I} + \tilde{\mathbf{P}}^{(-k)} \,
\, \mathbf{M}^{(k)} \,
\tilde{\mathbf{P}}^{(k)} \\
&= \mathbf{I} + \tilde{\mathbf{P}}^{(-k)} \,
\mathbf{t}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \,
\tilde{\mathbf{P}}^{(k)}
\end{split} \: .
$$

Notice that the matrix $\tilde{\mathbf{P}}^{(-k)}$ swaps the elements $\left[ \, k+1 \, : \, \right]$ of $\mathbf{t}^{(k)}$, resulting in a new vector $\tilde{\mathbf{t}}^{(k)} = \tilde{\mathbf{P}}^{(-k)} \, \mathbf{t}^{(k)}$. Similarly, the matrix $\tilde{\mathbf{P}}^{(k)}$ swaps the elements $\left[ \, k+1 \, : \, \right]$ of $\left( \mathbf{u}^{(k)} \right)^{\top}$ and, consequently, $\left( \mathbf{u}^{(k)} \right)^{\top} \, \tilde{\mathbf{P}}^{(k)} = \left( \mathbf{u}^{(k)} \right)^{\top}$. Then, the matrix $\tilde{\mathbf{Q}}^{(-k)}$ can be rewritten as follows:

$$
\tilde{\mathbf{Q}}^{(-k)} = \mathbf{I} + \tilde{\mathbf{t}}^{(k)} \otimes \left( \mathbf{u}^{(k)} \right)^{\top} \: .
$$

By using this result, we can verify that the matrix $\tilde{\mathbf{Q}}^{-1}$ is given by:

$$
\begin{split}
\tilde{\mathbf{Q}}^{-1}
&= \left[ \mathbf{I} + \tilde{\mathbf{t}}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} \right]
   \left[ \mathbf{I} + \tilde{\mathbf{t}}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \right] \\
&= \mathbf{I} + \tilde{\mathbf{t}}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} +
   \tilde{\mathbf{t}}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} + 
   \tilde{\mathbf{t}}^{(1)} \otimes 
   \underbrace{ \left( \mathbf{u}^{(1)} \right)^{\top} \tilde{\mathbf{t}}^{(2)}}_{= \, 0}
   \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \\
&= \mathbf{I} + \tilde{\mathbf{t}}^{(1)} \otimes \left( \mathbf{u}^{(1)} \right)^{\top} +
   \tilde{\mathbf{t}}^{(2)} \otimes \left( \mathbf{u}^{(2)} \right)^{\top} \\
&= 
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
\tilde{t}_{2}^{(1)} & 0 & 0 \\
\tilde{t}_{3}^{(1)} & 0 & 0
\end{array} \right] +
\left[ \begin{array}{ccc}
0 & 0 & 0 \\
0 & 0 & 0 \\
0 & \tilde{t}_{3}^{(2)} & 0
\end{array} \right] \\
&=
\left[ \begin{array}{ccc}
1 & 0 & 0 \\
\tilde{t}_{2}^{(1)} & 1 & 0 \\
\tilde{t}_{3}^{(1)} & \tilde{t}_{3}^{(2)} & 1
\end{array} \right]
\end{split} \: ,
$$

which is a unit **L**ower triangular matrix containing the permuted Gauss multipliers. Notice that this matrix $\tilde{\mathbf{Q}}^{-1}$ differs from that one presented in the previous class because the Gauss multipliers are permuted. Similarly to the previous class, the **U**pper triangular matrix $\mathbf{A}^{(2)}$ is defined as $\mathbf{U}$ and the original matrix $\mathbf{A}$ is factored as follows:

$$
\mathbf{P} \, \mathbf{A} = \tilde{\mathbf{L}} \, \mathbf{U} \: ,
$$

where $\mathbf{P}$ is the product of all permutation matrices. Notice that the lower triangular matrix $\tilde{\mathbf{L}}$ presented here is different from the lower triangular matrix $\mathbf{L}$ presented in the previous notebook.

The mathematical development presented here is valid for $3 \times 3$ matrices, however, it can be easily generalized to $N \times N$ matrices as well.

The LU decomposition described above can be implemented as follows:

    N = y.size
    C = np.copy(A)
    P = np.identity(N)
    
    for i = 1:N-1

        # permutation step (computation of C tilde)
        p, C = permut(C, i)
        P = P[p]
        
        # assert the pivot is nonzero
        assert C[i,i] != 0., 'null pivot!'
        
        # calculate the Gauss multipliers and store them 
        # in the lower part of C
        C[i+1:,i] = C[i+1:,i]/C[i,i]
        
        # zeroing of the elements in the ith column
        C[i+1:,i+1:] = C[i+1:,i+1:] - outer(C[i+1:,i], C[i,i+1:])
        
    return P, C

The permutation function can be defined as follows:

    permut (C, i):
        p = [j for j in range(C.shape[0])]
        imax = i + np.argmax(np.abs(C[i:,i]))
        if imax != i:
            p[i], p[imax] = p[imax], p[i]
    return p, C[p,:]

This algorithm receives a square matrix $\mathbf{A}$ and returns the permutation matrix $\mathbf{P}$ and a matrix $\mathbf{C}$ containing the triangular matrices $\mathbf{L}$ and $\mathbf{U}$. The elements of $\mathbf{L}$, except the unitary elements of its main diagonal, are stored below the main diagonal of $\mathbf{C}$. The elements of $\mathbf{U}$ are stored in the upper part of $\mathbf{C}$, including its main diagonal.

## Solving a linear system by using the LU decomposition with partial pivoting

Once the permutation matrix $\mathbf{P}$ and the triangular matrices $\tilde{\mathbf{L}}$ and $\mathbf{U}$ are calculated, we may use them to solve a linear system $\mathbf{A} \mathbf{x} = \mathbf{y}$. Let's first substitute the LU decomposition into the linear system:

$$
\begin{split}
\mathbf{A} \mathbf{x} &= \mathbf{y} \\
\mathbf{P} \mathbf{A} \mathbf{x} &= \mathbf{P} \mathbf{y} \\
\tilde{\mathbf{L}} \mathbf{U} \mathbf{x} &= \mathbf{P} \mathbf{y}
\end{split} \: .
$$

This equation shows that the original linear system can be represented by two triangular systems:

$$
\begin{split}
\tilde{\mathbf{L}}\mathbf{w} &= \mathbf{P} \mathbf{y} \\
\mathbf{U}\mathbf{x} &= \mathbf{w}
\end{split} \: ,
$$

where $\mathbf{w}$ is a dummy variable. Therefore, the linear system can be solved in two steps: 1) solve the lower triangular system for $\mathbf{w}$ and then 2) use it to solve the upper triangular system for $\mathbf{x}$.

## Computing inverses by using the LU decomposition

As we have learned in a previous class, each column of the inverse of a matrix can be computed by solving a linear system. It means that computing the inverse of an $N \times N$ matrix requires the solution of $N$ linear systems. Note that, by using the Gaussian elimination (with or without partial pivoting), we need to compute the triangular matrix just once, but a different 'data vector' for each one of the $N$ columns of the inverse matrix. If we instead use the LU decomposition (with or without partial pivoting), we need to compute the matrices $\mathbf{L}$ and $\mathbf{U}$ just once.

### Exercise

In your `my_functions.py` file:

1) Implement the LU decomposition with partial pivoting presented above. The code must receive a matrix `A` and return the matrices `C` and `P`.

2) Use the matrices `C` and `P` to solve a linear system `Ax = y`. The code must use the function you have created in the item 1, as well as the functions you have created for solving triangular systems.

3) Create a test that defines a square matrix `A`, compute the matrices `C` and `P` by using the function implemented in the item 1, use `C` to create the triangular matrices `L` and `U` and verify the condition `PA = LU` is satisfied.

In your `test_my_functions.py` file:

4) Create a test to compare the solution of the linear system  obtained by the function implemented in item 2 and the solution obtained by [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html).

5) Create a test to compare the matrices `P`, `L`, `U` obtainted by using your function with the matrices `P`, `L`, `U` obtainted by using the function [`scipy.linalg.lu`](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.linalg.lu.html). Be careful! The matrix `P` calculated by the routine `scipy.linalg.lu` is equal to the transpose of the matrix `P` calculated by the algorithm presented here.

In [9]:
from scipy.linalg import lu

In [10]:
N = 5
A = np.reshape(10.*np.random.rand(N*N), (N,N))

In [11]:
P_sp, L_sp, U_sp = lu(A)

In [12]:
np.allclose(A, np.dot(P_sp, np.dot(L_sp,U_sp)))

True